# Predicting authorized_flag in transactions_known

### Notebook automatically generated from your model

Model Random forest, trained on 2021-06-30 14:56:20.

#### Generated on 2022-06-13 10:15:07.332261

prediction
This notebook will reproduce the steps for a BINARY_CLASSIFICATION on  transactions_known.
The main objective is to predict the variable authorized_flag

#### Warning

The goal of this notebook is to provide an easily readable and explainable code that reproduces the main steps
of training the model. It is not complete: some of the preprocessing done by the DSS visual machine learning is not
replicated in this notebook. This notebook will not give the same results and model performance as the DSS visual machine
learning model.

Let's start with importing the required libs :

In [1]:
import sys
import dataiku
import numpy as np
import pandas as pd
import sklearn as sk
import dataiku.core.pandasutils as pdu
from dataiku.doctor.preprocessing import PCA
from collections import defaultdict, Counter

And tune pandas display options:

In [2]:
pd.set_option('display.width', 3000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

#### Importing base data

The first step is to get our machine learning dataset:

In [3]:
# We apply the preparation that you defined. You should not modify this.
preparation_steps = []
preparation_output_schema = {'columns': [{'name': 'transaction_id', 'type': 'bigint'}, {'name': 'authorized_flag', 'type': 'bigint'}, {'name': 'purchase_date', 'type': 'string'}, {'name': 'purchase_date_parsed', 'type': 'date'}, {'name': 'purchase_year', 'type': 'bigint'}, {'name': 'purchase_month', 'type': 'bigint'}, {'name': 'purchase_day', 'type': 'bigint'}, {'name': 'purchase_dow', 'type': 'bigint'}, {'name': 'purchase_weekend', 'type': 'bigint'}, {'name': 'purchase_hour', 'type': 'bigint'}, {'name': 'card_id', 'type': 'string'}, {'name': 'merchant_id', 'type': 'string'}, {'name': 'merchant_category_id', 'type': 'bigint'}, {'name': 'item_category', 'type': 'string'}, {'name': 'purchase_amount', 'type': 'double'}, {'name': 'signature_provided', 'type': 'bigint'}, {'name': 'card_first_active_month', 'type': 'string'}, {'name': 'card_first_active_month_parsed', 'type': 'date'}, {'name': 'days_active', 'type': 'bigint'}, {'name': 'card_reward_program', 'type': 'string'}, {'name': 'card_latitude', 'type': 'double'}, {'name': 'card_longitude', 'type': 'double'}, {'name': 'card_fico_score', 'type': 'bigint'}, {'name': 'card_age', 'type': 'bigint'}, {'name': 'merchant_subsector_description', 'type': 'string'}, {'name': 'merchant_latitude', 'type': 'double'}, {'name': 'merchant_longitude', 'type': 'double'}, {'name': 'merchant_location', 'type': 'string'}, {'name': 'merchant_state', 'type': 'string'}, {'name': 'merchant_state_enName', 'type': 'string'}, {'name': 'card_location', 'type': 'string'}, {'name': 'merchant_cardholder_distance', 'type': 'double'}, {'name': 'card_state', 'type': 'string'}, {'name': 'card_state_enName', 'type': 'string'}], 'userModified': False}

ml_dataset_handle = dataiku.Dataset('transactions_known')
ml_dataset_handle.set_preparation_steps(preparation_steps, preparation_output_schema)
%time ml_dataset = ml_dataset_handle.get_dataframe(limit = 100000)

print ('Base data has %i rows and %i columns' % (ml_dataset.shape[0], ml_dataset.shape[1]))
# Five first records",
ml_dataset.head(5)

CPU times: user 978 ms, sys: 90.5 ms, total: 1.07 s
Wall time: 3.77 s
Base data has 100000 rows and 34 columns


,transaction_id,authorized_flag,purchase_date,purchase_date_parsed,purchase_year,purchase_month,purchase_day,purchase_dow,purchase_weekend,purchase_hour,card_id,merchant_id,merchant_category_id,item_category,purchase_amount,signature_provided,card_first_active_month,card_first_active_month_parsed,days_active,card_reward_program,card_latitude,card_longitude,card_fico_score,card_age,merchant_subsector_description,merchant_latitude,merchant_longitude,merchant_location,merchant_state,merchant_state_enName,card_location,merchant_cardholder_distance,card_state,card_state_enName
0,73553,1,2017-06-03 08:56:44,2017-06-03 08:56:44,2017,6,3,6,1,8,C_ID_b72230e635,M_ID_120024bfc9,276,B,195.99,0,2016-12,2016-12-01,184,cash_back,41.669,-73.920,636,97,consumer electronics,41.705,-73.951,POINT(-73.951 41.705),New York,New York,POINT(-73.92 41.669),4.76,New York,New York
1,81534,1,2017-06-15 20:58:47,2017-06-15 20:58:47,2017,6,15,4,0,20,C_ID_b72230e635,M_ID_13cb138513,198,A,1191.91,0,2016-12,2016-12-01,196,cash_back,41.669,-73.920,636,97,insurance,42.668,-73.808,POINT(-73.808 42.668),New York,New York,POINT(-73.92 41.669),111.47,New York,New York
2,32505,1,2017-03-15 16:55:33,2017-03-15 16:55:33,2017,3,15,3,0,16,C_ID_b7224e1512,M_ID_2637773dd2,422,B,266.60,0,2017-02,2017-02-01,42,dining_entertainment,43.211,-71.577,544,28,consumer electronics,43.226,-71.544,POINT(-71.544 43.226),New Hampshire,New Hampshire,POINT(-71.577 43.211),3.15,New Hampshire,New Hampshire
3,64039,1,2017-05-16 18:25:50,2017-05-16 18:25:50,2017,5,16,2,0,18,C_ID_b7224e1512,M_ID_f2ae17fdd2,884,A,84.13,0,2017-02,2017-02-01,104,dining_entertainment,43.211,-71.577,544,28,consumer electronics,41.743,-73.938,POINT(-73.938 41.743),New York,New York,POINT(-71.577 43.211),253.24,New Hampshire,New Hampshire
4,57791,1,2017-05-05 03:13:51,2017-05-05 03:13:51,2017,5,5,5,0,3,C_ID_b72498b35a,M_ID_e5374dabc0,130,D,8.72,0,2017-04,2017-04-01,34,dining_entertainment,41.710,-73.974,665,25,luxury goods,43.218,-71.526,POINT(-71.526 43.218),New Hampshire,New Hampshire,POINT(-73.974 41.71),261.59,New York,New York


#### Initial data management

The preprocessing aims at making the dataset compatible with modeling.
At the end of this step, we will have a matrix of float numbers, with no missing values.
We'll use the features and the preprocessing steps defined in Models.

Let's only keep selected features

In [0]:
ml_dataset = ml_dataset[['card_fico_score', 'signature_provided', 'card_age', 'merchant_subsector_description', 'purchase_amount', 'merchant_state', 'authorized_flag']]

Let's first coerce categorical columns into unicode, numerical features into floats.

In [0]:
# astype('unicode') does not work as expected

def coerce_to_unicode(x):
    if sys.version_info < (3, 0):
        if isinstance(x, str):
            return unicode(x,'utf-8')
        else:
            return unicode(x)
    else:
        return str(x)


categorical_features = ['signature_provided', 'merchant_subsector_description', 'merchant_state']
numerical_features = ['card_fico_score', 'card_age', 'purchase_amount']
text_features = []
from dataiku.doctor.utils import datetime_to_epoch
for feature in categorical_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in text_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in numerical_features:
    if ml_dataset[feature].dtype == np.dtype('M8[ns]') or (hasattr(ml_dataset[feature].dtype, 'base') and ml_dataset[feature].dtype.base == np.dtype('M8[ns]')):
        ml_dataset[feature] = datetime_to_epoch(ml_dataset[feature])
    else:
        ml_dataset[feature] = ml_dataset[feature].astype('double')

We are now going to handle the target variable and store it in a new variable:

In [0]:
target_map = {'0': 0, '1': 1}
ml_dataset['__target__'] = ml_dataset['authorized_flag'].map(str).map(target_map)
del ml_dataset['authorized_flag']


# Remove rows for which the target is unknown.
ml_dataset = ml_dataset[~ml_dataset['__target__'].isnull()]

ml_dataset['__target__'] = ml_dataset['__target__'].astype(np.int64)

#### Cross-validation strategy

The dataset needs to be split into 2 new sets, one that will be used for training the model (train set)
and another that will be used to test its generalization capability (test set)

Important note: your model used a more advanced cross-validation strategy.
For the purpose of this notebook, it has been simplified to a random split of
a single dataset

In [0]:
train, test = pdu.split_train_valid(ml_dataset, prop=0.8)
print ('Train data has %i rows and %i columns' % (train.shape[0], train.shape[1]))
print ('Test data has %i rows and %i columns' % (test.shape[0], test.shape[1]))

#### Features preprocessing

The first thing to do at the features level is to handle the missing values.
Let's reuse the settings defined in the model

In [0]:
drop_rows_when_missing = []
impute_when_missing = [{'feature': 'card_fico_score', 'impute_with': 'MEAN'}, {'feature': 'signature_provided', 'impute_with': 'MODE'}, {'feature': 'card_age', 'impute_with': 'MEAN'}, {'feature': 'purchase_amount', 'impute_with': 'MEAN'}]

# Features for which we drop rows with missing values"
for feature in drop_rows_when_missing:
    train = train[train[feature].notnull()]
    test = test[test[feature].notnull()]
    print ('Dropped missing records in %s' % feature)

# Features for which we impute missing values"
for feature in impute_when_missing:
    if feature['impute_with'] == 'MEAN':
        v = train[feature['feature']].mean()
    elif feature['impute_with'] == 'MEDIAN':
        v = train[feature['feature']].median()
    elif feature['impute_with'] == 'CREATE_CATEGORY':
        v = 'NULL_CATEGORY'
    elif feature['impute_with'] == 'MODE':
        v = train[feature['feature']].value_counts().index[0]
    elif feature['impute_with'] == 'CONSTANT':
        v = feature['value']
    train[feature['feature']] = train[feature['feature']].fillna(v)
    test[feature['feature']] = test[feature['feature']].fillna(v)
    print ('Imputed missing values in feature %s with value %s' % (feature['feature'], coerce_to_unicode(v)))

We can now handle the categorical features (still using the settings defined in Models):

Let's dummy-encode the following features.
A binary column is created for each of the 100 most frequent values.

In [0]:
LIMIT_DUMMIES = 100

categorical_to_dummy_encode = ['signature_provided', 'merchant_subsector_description']

# Only keep the top 100 values
def select_dummy_values(train, features):
    dummy_values = {}
    for feature in categorical_to_dummy_encode:
        values = [
            value
            for (value, _) in Counter(train[feature]).most_common(LIMIT_DUMMIES)
        ]
        dummy_values[feature] = values
    return dummy_values

DUMMY_VALUES = select_dummy_values(train, categorical_to_dummy_encode)

def dummy_encode_dataframe(df):
    for (feature, dummy_values) in DUMMY_VALUES.items():
        for dummy_value in dummy_values:
            dummy_name = u'%s_value_%s' % (feature, coerce_to_unicode(dummy_value))
            df[dummy_name] = (df[feature] == dummy_value).astype(float)
        del df[feature]
        print ('Dummy-encoded feature %s' % feature)

dummy_encode_dataframe(train)

dummy_encode_dataframe(test)

Let's impact code the following featuress.

In [0]:
categorical_to_impact_code = ['merchant_state']

# impact coding
from dataiku.notebook import ImpactCoding

impact_coding = ImpactCoding('multiple', m=10)  # tune m
train = pd.concat([train, impact_coding.fit_transform(train[categorical_to_impact_code], train['__target__'])], axis=1)
for feature in categorical_to_impact_code:
    del train[feature]

test = pd.concat([test, impact_coding.transform(test[categorical_to_impact_code])], axis=1)
for feature in categorical_to_impact_code:
    del test[feature]

for feature in categorical_to_impact_code:
    print ('Impact Coded feature %s ' % feature)

Let's rescale numerical features

In [0]:
rescale_features = {'card_fico_score': 'AVGSTD', 'card_age': 'AVGSTD', 'purchase_amount': 'AVGSTD'}
for (feature_name, rescale_method) in rescale_features.items():
    if rescale_method == 'MINMAX':
        _min = train[feature_name].min()
        _max = train[feature_name].max()
        scale = _max - _min
        shift = _min
    else:
        shift = train[feature_name].mean()
        scale = train[feature_name].std()
    if scale == 0.:
        del train[feature_name]
        del test[feature_name]
        print ('Feature %s was dropped because it has no variance' % feature_name)
    else:
        print ('Rescaled %s' % feature_name)
        train[feature_name] = (train[feature_name] - shift).astype(np.float64) / scale
        test[feature_name] = (test[feature_name] - shift).astype(np.float64) / scale

#### Modeling

Before actually creating our model, we need to split the datasets into their features and labels parts:

In [0]:
X_train = train.drop('__target__', axis=1)
X_test = test.drop('__target__', axis=1)

y_train = np.array(train['__target__'])
y_test = np.array(test['__target__'])

Now we can finally create our model!

In [0]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=225,
    random_state=1337,
    max_depth=18,
    min_samples_leaf=1,
    verbose=2)

We set "class_weight" as the weighting strategy:

In [0]:
clf.class_weight = "balanced"

... And train the model

In [0]:
%time clf.fit(X_train, y_train)

Build up our result dataset

The model is now trained, we can apply it to our test set:

In [0]:
%time _predictions = clf.predict(X_test)
%time _probas = clf.predict_proba(X_test)
predictions = pd.Series(data=_predictions, index=X_test.index, name='predicted_value')
cols = [
    u'probability_of_value_%s' % label
    for (_, label) in sorted([(int(target_map[label]), label) for label in target_map])
]
probabilities = pd.DataFrame(data=_probas, index=X_test.index, columns=cols)

# Build scored dataset
results_test = X_test.join(predictions, how='left')
results_test = results_test.join(probabilities, how='left')
results_test = results_test.join(test['__target__'], how='left')
results_test = results_test.rename(columns= {'__target__': 'authorized_flag'})

Let's have a look at feature importances

In [0]:
feature_importances_data = []
features = X_train.columns
for feature_name, feature_importance in zip(features, clf.feature_importances_):
    feature_importances_data.append({
        'feature': feature_name,
        'importance': feature_importance
    })

# Plot the results
pd.DataFrame(feature_importances_data)\
    .set_index('feature')\
    .sort_values(by='importance')[-10::]\
    .plot(title='Top 10 most important variables',
          kind='barh',
          figsize=(10, 6),
          color='#348ABD',
          alpha=0.6,
          lw='1',
          edgecolor='#348ABD',
          grid=False,)

#### Results

You can measure the model's accuracy:

In [0]:
from dataiku.doctor.utils.metrics import mroc_auc_score
y_test_ser = pd.Series(y_test)
 
print ('AUC value:', mroc_auc_score(y_test_ser, _probas))

We can also view the predictions directly.
Since scikit-learn only predicts numericals, the labels have been mapped to 0,1,2 ...
We need to 'reverse' the mapping to display the initial labels.

In [0]:
inv_map = { target_map[label] : label for label in target_map}
predictions.map(inv_map)

That's it. It's now up to you to tune your preprocessing, your algo, and your analysis !
